In [1]:
# grab data to do features predictive of 400mM alcohol worms
# Conny Lin | May 20, 2020
# -----------------------------------------------------------------
# Try KNN:
#     X = curve and another measure from drunkposture2.dat
#     y = 0mM(0) or 400mM(1) 
# -----------------------------------------------------------------
# These ones could be useful:
# *  'N2_5x3_t96h20C_100s30x10s10s_C0109bb.drunkposture2.dat'
#     * odrunkposture2 = '-O drunkposture2 -o nNslwakbcemM';
# *  'N2_5x3_t96h20C_100s30x10s10s_C0109bb.trv',
#     * revbeethoven_trv = '--plugin MeasureReversal::tap::dt=1::collect=0.5::postfix=trv';
# *  'N2_5x3_t96h20C_100s30x10s10s_C0109bb.shanespark.dat',
#     * oshanespark = '-O shanespark -o nNss*b12M'; % standard for Beethoven
# *  'trinitySummary.mat',
#     * otrinity = '-O trinity -N all -o nNss*b12xyMmeSakcr'; 
#     * stored as a variable masterData, first column is worm id in string, second column is the trinity output.
# *  'Gangnam.mat'
#     * ogangnam = '-O gangnam -N all -o DpmcobdPsSruvxy1';
# -----------------------------------------------------------------

In [85]:
# define local variables
dir_save = '/Users/connylin/Dropbox/CA/ED _20200119 Brain Station Data Science Diploma/Capstone/data'
path_MWTDBcsv = '/Volumes/COBOLT/MWT/MWTDB.csv'
output_csv_name = 'drunkposture2_90s_95s_mean_KNN.csv'

In [22]:
import os
import glob
import pickle
from pathlib import PurePath
import numpy as np
import pandas as pd

In [3]:
# load database
MWTDB = pd.read_csv(os.path.join(path_MWTDBcsv))

In [4]:
# get N2 vs N2_400mM data
ind = (MWTDB['groupname'] == 'N2') | (MWTDB['groupname'] == 'N2_400mM')
# check if output is correct
print('groups included:')
print(MWTDB.loc[ind,'groupname'].unique())
# reduce database
MWTDB = MWTDB.loc[ind,:].copy()

groups included:
['N2' 'N2_400mM']


In [5]:
MWTDB.shape

(1975, 15)

In [6]:
# reindex
MWTDB.reset_index(drop=True,inplace=True)
MWTDB.head()

,mwtid,mwtname,mwtpath,expname,exp_date,tracker,expter,groupname,strain,rx,rc,preplate,ISI,postrec,tapN
0,134,20110602_195554,/Volumes/COBOLT/MWT/20110602A_PO_100s30x10s10s...,20110602A_PO_100s30x10s10s,20110602,A,PO,N2,N2,NaN,100s30x10s10s,100,10,10,30
1,135,20110602_200308,/Volumes/COBOLT/MWT/20110602A_PO_100s30x10s10s...,20110602A_PO_100s30x10s10s,20110602,A,PO,N2,N2,NaN,100s30x10s10s,100,10,10,30
2,224,20110711_063715,/Volumes/COBOLT/MWT/20110711A_PO_100s30x10s10s...,20110711A_PO_100s30x10s10s_InnexinScreenDay6B,20110711,A,PO,N2,N2,NaN,100s30x10s10s,100,10,10,30
3,225,20110711_064416,/Volumes/COBOLT/MWT/20110711A_PO_100s30x10s10s...,20110711A_PO_100s30x10s10s_InnexinScreenDay6B,20110711,A,PO,N2,N2,NaN,100s30x10s10s,100,10,10,30
4,226,20110711_065114,/Volumes/COBOLT/MWT/20110711A_PO_100s30x10s10s...,20110711A_PO_100s30x10s10s_InnexinScreenDay6B,20110711,A,PO,N2,N2,NaN,100s30x10s10s,100,10,10,30


In [66]:
# get drunkposture2.dat data
# !TODO! many files do not have drunkposture2.dat files. Run MWT 

# define variables from drunposture2 to use
y_varnames_drunkposture = ['curve','speed']
# load column names
with open(os.path.join(dir_save,'legend_drunkposture2.pickle'),'rb') as f:
    column_names = pickle.load(f)

# get all mwt paths
mwtpaths_all = MWTDB['mwtpath'].values

# initialize output array
data = np.empty((len(mwtpaths_all),len(column_names)))
data[:] = np.NaN

# get drunkposture.dat data
# combine multiple means - # get mean values from 90-95s recording
for mwtpath, row_number in zip(mwtpaths_all,range(0,data.shape[0])):
    #iteration reporter
    if (row_number%100==0):
        print(f'iterating {row_number}th files')
        
    # get  drunkposture2.dat under this mwt folder
    files = glob.glob(f'{mwtpath}/*drunkposture2.dat')
    # get file names only
    filename = list(map(lambda x: x.name, list(map(PurePath,files))))
    # check if there is only one file
    if len(filename)==0:
        print(f'\t{row_number}: {mwtpath} \n\t no drunkposture2.dat file found')
    elif len(filename)>1:
        print(f'\t{row_number}: {mwtpath} \n\t more than one drunkposture2.dat file found')
    else:
        # the first row of data is loaded as header, but the first row of data usually are very noisy and never used for analysis. So it's ok to lose the data.
        df = pd.read_csv(files[0],delim_whitespace=True)

        # get files that are between 90-95s
        i = (df.iloc[:,0] < 95) & (df.iloc[:,0] > 90)
        df_short = df.loc[i,:].copy()
        df_short.columns = column_names['name']
        # declare var
        mean_data = []
        # get the mean values of all the measures between 90-95s
        mean_data = df_short.mean().values
        # check data for integrity
        if len(mean_data)==len(column_names):
            data[row_number,:] = mean_data # put in data
        else:
            display(mean_data)
            print(f'\t{rown_number}: {mwtpath}\n\t something wrong with data')
print('complete')

iterating 0th files
iterating 100th files
	158: /Volumes/COBOLT/MWT/20120403C_BM_100s30x60s10s/N2/20120403_200237 
	 no drunkposture2.dat file found
	159: /Volumes/COBOLT/MWT/20120403C_BM_100s30x60s10s/N2/20120403_211607 
	 no drunkposture2.dat file found
	160: /Volumes/COBOLT/MWT/20120403C_BM_100s30x60s10s/N2/20120403_222758 
	 no drunkposture2.dat file found
iterating 200th files
iterating 300th files
	321: /Volumes/COBOLT/MWT/20121106B_JX_900s0x0s0s_15min/N2/20121106_154522 
	 no drunkposture2.dat file found
	326: /Volumes/COBOLT/MWT/20121106B_JX_900s0x0s0s_15min/N2_400mM/20121106_160057 
	 more than one drunkposture2.dat file found
	327: /Volumes/COBOLT/MWT/20121106B_JX_900s0x0s0s_15min/N2_400mM/20121106_163514 
	 no drunkposture2.dat file found
iterating 400th files
iterating 500th files
iterating 600th files
iterating 700th files
	707: /Volumes/COBOLT/MWT/20130919C_CL_3600s0x0s0s_60minBasalInverted/N2/20130919_100507 
	 no drunkposture2.dat file found
iterating 800th files
	817: 

In [86]:
d = pd.DataFrame(data,columns=column_names['name'])
dp = pd.DataFrame(mwtpaths_all,columns=['mwtpath'])
df = pd.concat([dp,d],axis=1)
df.to_csv(os.path.join(dir_save, output_csv_name))

In [68]:
# put in data frame
# do KNN